In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv("../data/train_small.csv")
test = pd.read_csv("../data/test_fe.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_small.csv'

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
x_train_use, y_train_use= train.drop('target', axis=1), train['target']

In [ ]:
x_train,x_val, y_train, y_val=train_test_split(x_train_use, y_train_use, train_size=0.8, random_state=42)

In [ ]:
x_train.shape

### Random Forest Classifier

In [ ]:
param_grid = {
    'max_depth': list(range(3, 7)),  
    'max_features': list(range(3, 7)),
    'n_estimators': [10,20,30,40,50]
}


In [ ]:
rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)


In [ ]:
grid_search.fit(x_train, y_train)

In [ ]:
best_param=grid_search.best_params_
best_model=grid_search.best_estimator_

In [ ]:
best_param

In [ ]:
best_model.fit(x_train, y_train)

In [ ]:
y_pred=best_model.predict(x_train)

In [ ]:
print(classification_report(y_train, y_pred))

In [ ]:
def plot_confusion_matrix(title,conf_matrix):
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', cbar=False,
                xticklabels=['Predicted 0', 'Predicted 1'],
                yticklabels=['Actual 0', 'Actual 1'])
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(title)
    plt.savefig("../conf/"+title+".png")
    plt.show()

In [ ]:
conf=confusion_matrix(y_train, y_pred)
sns.heatmap(conf, annot=True, cmap= "Blues")

In [ ]:
#plot_confusion_matrix("Random_Forest_Classfier_Train",conf)

In [ ]:
y_pred_val=best_model.predict(x_val)

In [ ]:
print(classification_report(y_val, y_pred_val))

In [ ]:
conf_val=confusion_matrix(y_val, y_pred_val)

In [ ]:
#plot_confusion_matrix("Random_Forest_Classfier_Validate",conf_val)
sns.heatmap(conf_val, annot=True, cmap= "Blues")

### Extract Result

In [ ]:
import time

start=time.time()

best_model.fit(x_train_use, y_train_use)

elapsed=time.time()-start

In [ ]:
pred_use= best_model.predict(x_train_use)

In [ ]:
cross_val_score(best_model, x_train_use, y_train_use, scoring='f1')

In [ ]:
print(classification_report(y_train_use, pred_use))

In [ ]:
f1_train = f1_score(y_train_use, pred_use)
prec_train = precision_score(y_train_use, pred_use)
recall_train = recall_score(y_train_use, pred_use)

In [ ]:
x_test, y_test= test.drop('target', axis=1), test['target']

In [ ]:
pred_test=best_model.predict(x_test)

In [ ]:
print(classification_report(y_test, pred_test))

In [ ]:
f1_test = f1_score(y_test, pred_test)
prec_test = precision_score(y_test, pred_test)
recall_test = recall_score(y_test, pred_test)

In [ ]:
conf_test=confusion_matrix(y_test, pred_test)

In [ ]:
plot_confusion_matrix("rf",conf_test)

In [ ]:
df=pd.DataFrame()
df["f1_test"] = [f1_test]
df["f1_train"] = f1_train
df["prec_train"] = prec_train
df["prec_test"] = prec_test
df["recall_train"] = recall_train
df["recall_test"] = recall_test
df["model"] = "Random Forest"
df["elapsed"] = elapsed

In [ ]:
df.head()

In [ ]:
df.to_csv("../results/regular/rf.csv", index=False)